In [1]:
from dotenv import load_dotenv
import sys

sys.path.append("../common")
load_dotenv()

True

In [2]:
import os
from langsmith_tracker import set_tracking
from langchain_print import stream_response

# 인스턴스를 생성할 때 필요한 매개변수를 전달합니다.
set_tracking(project_name="03.OutputParser")

Langsmith 추적이 활성화되었습니다. [프로젝트명: 03.OutputParser]


### DatetimeOutputParser

LLM의 출력을 datetime형식으로 출력하는 파서입니다.

| **형식 코드** | **설명**                 | **예시**             |
|---------------|--------------------------|----------------------|
| `%Y`          | 4자리 연도               | 2024                |
| `%y`          | 2자리 연도               | 24                  |
| `%m`          | 2자리 월 (01~12)         | 11                  |
| `%d`          | 2자리 일 (01~31)         | 25                  |
| `%H`          | 24시간 형식의 시 (00~23) | 14                  |
| `%I`          | 12시간 형식의 시 (01~12) | 02                  |
| `%p`          | AM/PM                    | PM                  |
| `%M`          | 분 (00~59)               | 30                  |
| `%S`          | 초 (00~59)               | 45                  |
| `%f`          | 마이크로초 (000000~999999)| 123456              |
| `%a`          | 요일의 짧은 이름         | Mon                 |
| `%A`          | 요일의 전체 이름         | Monday              |
| `%w`          | 요일을 숫자로 표시 (0~6, 일요일이 0) | 1 |
| `%b`          | 월의 짧은 이름           | Nov                 |
| `%B`          | 월의 전체 이름           | November            |
| `%U`          | 연중 주 번호 (일요일 시작, 00~53) | 47 |
| `%W`          | 연중 주 번호 (월요일 시작, 00~53) | 47 |
| `%c`          | 로컬 날짜 및 시간 표시    | Mon Nov 25 14:30:45 2024 |
| `%x`          | 로컬 날짜 표시           | 11/25/24            |
| `%X`          | 로컬 시간 표시           | 14:30:45            |
| `%j`          | 연중 일 번호 (001~366)   | 329                 |
| `%z`          | UTC 오프셋               | +0900               |
| `%Z`          | 시간대 이름              | KST                 |
| `%%`          | `%` 문자 자체            | %                   |


In [8]:
from langchain.output_parsers import DatetimeOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 날짜 및 시간 출력 파서
output_parser = DatetimeOutputParser()
output_parser.format = "%Y-%m-%d"

# 사용자 질문에 대한 답변 템플릿
template = """Answer the users question:\n\n#Format Instructions: \n{format_instructions}\n\n#Question:\n{question}\n\n#Answer: """

prompt = PromptTemplate.from_template(
    template=template,
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

prompt

PromptTemplate(input_variables=['question'], input_types={}, partial_variables={'format_instructions': "Write a datetime string that matches the following pattern: '%Y-%m-%d'.\n\nExamples: 1395-04-20, 1172-11-17, 1618-11-21\n\nReturn ONLY this string, no other words!"}, template='Answer the users question:\n\n#Format Instructions: \n{format_instructions}\n\n#Question:\n{question}\n\n#Answer: ')

In [14]:
llm = ChatOpenAI(model="gpt-4o", temperature=0)
chain = prompt | llm | output_parser

response = chain.invoke({"question": "이더리움의 출시일은?"})

response

datetime.datetime(2015, 7, 30, 0, 0)

In [12]:
response.strftime("%Y-%m-%d")

'2009-01-03'